In [ ]:
%load_ext autoreload
%autoreload 2

# Unbiased ECMWF

Here we propose a small model which is a debiased ECMWF forecast according to the data we have.
The plan is
* Compute the bias between the ECMWF model and the observations
* Make a debiased model
* Turn this model into a probabilistic forecast
* Score the forecast
For now we do it only on temperature and weeks 3-4.

In [ ]:
import dask
import dask.array as da
import dask.distributed
import dask_jobqueue
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import scipy.stats
import xarray as xr
import xskillscore as xs

In [ ]:
from typing import Tuple

In [ ]:
from crims2s.util import fix_dataset_dims

In [ ]:
INPUT_TRAIN = '***BASEDIR***training-input/0.3.0/netcdf'
OBSERVATIONS = '***BASEDIR***training-output-reference/'
BENCHNMARK = '***BASEDIR***training-output-benchmark/'

## Boost dask cluster

In [ ]:
def start_dask(user: str = "landryda", jobs: int = 2):
    
    if user == "landryda":
        cluster = dask_jobqueue.SLURMCluster(env_extra=['source ***HOME***.bash_profile','conda activate s2s'])
    elif user == "gierscjo":
        cluster = dask_jobqueue.SLURMCluster(cores=12,
                                             processes=6,
                                             memory='128G',
                                             env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
                                             name='agri-dask',
                                             local_directory='***CACHE***', # METTRE VOTRE LOGIN CRIM ICI
                                             walltime='3:00:00')
    
    cluster.scale(jobs=jobs)  # Scale to two working nodes as configured.
    
    return dask.distributed.Client(cluster)

In [ ]:
client = start_dask(user="gierscjo")

In [ ]:
client

## Generic Functions

In [ ]:
def extract_train_validation_from_lead_time(xr_data, time_slice: slice = slice('14D', '27D')) -> Tuple:
    
    xr_data_sub = xr_data.sel(lead_time=time_slice)
    xr_data_sub_train = xr_data_sub.sel(forecast_year=slice(None, 2018))
    xr_data_sub_val = xr_data_sub.sel(forecast_year=slice(2019, None))
    
    return xr_data_sub, xr_data_sub_train, xr_data_sub_val

In [ ]:
def compute_and_correct_bias(data_center_train, data_center_val, obs_train):
    
    bias = (obs_train - data_center_train).mean(dim=['lead_time', 'forecast_year'])
    corrected_bias = data_center_val + bias
    
    return bias, corrected_bias

## Read data

### ECMWF

In [ ]:
CENTER = 'ecmwf'
FIELD = 't2m'

In [ ]:
input_path = pathlib.Path(INPUT_TRAIN)

In [ ]:
input_files = sorted([f for f in input_path.iterdir() if CENTER in f.stem and FIELD in f.stem])

In [ ]:
input_files[:10]

In [ ]:
ecmwf = xr.open_mfdataset(input_files, preprocess=fix_dataset_dims)

In [ ]:
ecmwf_w34 = ecmwf.sel(lead_time=slice('14D', '27D'))
ecmwf_w34_train = ecmwf_w34.sel(forecast_year=slice(None, 2018))
ecmwf_w34_val = ecmwf_w34.sel(forecast_year=slice(2019, None))

ecmwf_w34, ecmwf_w34_train, ecmwf_w34_val = extract_train_validation_from_lead_time(ecmwf, time_slice=slice('14D', '27D'))

In [ ]:
ecmwf_w34_train

In [ ]:
ecmwf_w34_val

### Observations

In [ ]:
obs_path = pathlib.Path(OBSERVATIONS)
obs_files = [f for f in obs_path.iterdir() if 't2m' in f.stem]

In [ ]:
obs = xr.open_mfdataset(obs_files, preprocess=fix_dataset_dims).isel(lead_time=slice(1, None))

In [ ]:
obs_w34_train = obs_w34.sel(forecast_year=slice(None, 2018))
obs_w34_val = obs_w34.sel(forecast_year=slice(2019, None))

In [ ]:
obs_w34_val

## Debiasing

### Compute bias using training data

In [ ]:
ecmwf_w34_bias = (obs_w34_train - ecmwf_w34_train).mean(dim=['lead_time', 'forecast_year'])

### Bias correct ECMWF

In [ ]:
ecmwf_w34_val_corrected = ecmwf_w34_val + ecmwf_w34_bias

In [ ]:
ecmwf_w34_bias

### Get thresholds from train observations

In [ ]:
obs_w34_train_thresholds = obs_w34_train.quantile([0.33, 0.67], dim=['lead_time', 'forecast_year'])

In [ ]:
obs_w34_train_thresholds

### Compute p of thresholds according to the model

There are two ways to do this. 
We can either count the amount of members that are whithin each category.
Or compute a distribution of all the members of the model, and then compute the value of the CDF for each threshold.

Here we do it using the distribution method.

#### Compute a distribution of the members of the model

In [ ]:
ecmwf_w34_val_corrected_mean = ecmwf_w34_val_corrected.mean(dim=['realization', 'lead_time'])
ecmwf_w34_val_corrected_std = ecmwf_w34_val_corrected.std(dim=['realization', 'lead_time'])

#### Compute the value of the CDF for each threshold

In [ ]:
ecmwf_w34_val_corrected_mean

In [ ]:
def make_probabilistic(forecast, obs):
    thresholds = obs.quantile([0.33, 0.67], dim=['lead_time', 'forecast_year'])
    
    loc = forecast.mean(dim=['realization', 'lead_time']).compute().t2m
    scale = forecast.std(dim=['realization', 'lead_time']).compute().t2m
    
    cdfs = xr.apply_ufunc(scipy.stats.norm.cdf, thresholds.t2m, dask='allowed', kwargs={'loc': loc, 'scale': scale})
    
    below = cdfs.isel(quantile=0).drop_vars('quantile')
    normal = (cdfs.isel(quantile=1) - cdfs.isel(quantile=0))
    above = xr.ones_like(normal) - cdfs.isel(quantile=1).drop_vars('quantile')
    
    return xr.concat([below, normal, above], 'category').assign_coords(category=['below normal', 'near normal', 'above normal'])

In [ ]:
val_probabilistic_forecast = make_probabilistic(ecmwf_w34_val_corrected, obs_w34_train)

In [ ]:
val_probabilistic_forecast.isel(category=2, forecast_dayofyear=40).plot()

In [ ]:
val_probabilistic_forecast.isel(category=1, forecast_dayofyear=40).plot()

In [ ]:
loc = ecmwf_w34_val_corrected_mean.compute().t2m
scale = ecmwf_w34_val_corrected_std.compute().t2m

In [ ]:
cdfs = xr.apply_ufunc(scipy.stats.norm.cdf, obs_w34_train_thresholds.t2m, dask='allowed', kwargs={'loc': loc, 'scale': scale})

In [ ]:
cdfs.isel(quantile=1).plot()

In [ ]:
below = cdfs.isel(quantile=0).drop_vars('quantile')
normal = (cdfs.isel(quantile=1) - cdfs.isel(quantile=0))
above = xr.ones_like(normal) - cdfs.isel(quantile=1).drop_vars('quantile')

In [ ]:
val_probabilistic_forecast = xr.concat([below, normal, above], 'category', coords='minimal').assign_coords(category=['below normal', 'near normal', 'above normal'])

### Sanity check

In [ ]:
val_probabilistic_forecast.sum(dim='category').isel(forecast_dayofyear=0).plot()

## Score it

### Make a probabilistic version of the val obs

In [ ]:
obs_w34_val

In [ ]:
loc = obs_w34_val.mean(dim=['lead_time']).compute().t2m
scale = obs_w34_val.std(dim=['lead_time']).compute().t2m

In [ ]:
cdfs = xr.apply_ufunc(scipy.stats.norm.cdf, obs_w34_train_thresholds.t2m, dask='allowed', kwargs={'loc': loc, 'scale': scale})

In [ ]:
below = cdfs.isel(quantile=0).drop_vars('quantile')
normal = (cdfs.isel(quantile=1) - cdfs.isel(quantile=0))
above = xr.ones_like(normal) - cdfs.isel(quantile=1).drop_vars('quantile')

In [ ]:
val_probabilistic_obs = xr.concat([below, normal, above], 'category').assign_coords(category=['below normal', 'near normal', 'above normal'])

In [ ]:
val_probabilistic_obs.isel(category=2, forecast_dayofyear=0).plot()

### Compare both proba using rps

In [ ]:
val_probabilistic_obs

In [ ]:
xs.rps(val_probabilistic_obs, val_probabilistic_forecast, category_edges=None, input_distributions='p', dim=['forecast_dayofyear']).plot()

In [ ]:
val_probabilistic_obs.isel(forecast_dayofyear=0, category=2).plot()

In [ ]:
val_probabilistic_forecast.isel(forecast_dayofyear=0, category=2).plot()

## Process week 5-6

In [ ]:
ecmwf_w56, ecmwf_w56_train, ecmwf_w56_val = extract_train_validation_from_lead_time(ecmwf, time_slice=slice('28D', '41D'))

In [ ]:
obs_w56, obs_w56_train, obs_w56_val = extract_train_validation_from_lead_time(obs, time_slice=slice('28D', '41D'))

In [ ]:
ecmwf_w56_bias, ecmwf_w56_val_corrected = compute_and_correct_bias(data_center_train=ecmwf_w56_train,
                                                                   data_center_val=ecmwf_w56_val, 
                                                                   obs_train=obs_w56_train)

In [ ]:
obs_w56_train_thresholds = obs_w56_train.quantile([0.33, 0.66], dim=['lead_time', 'forecast_year'])

In [ ]:
ecmwf_w56_val_corrected_mean = ecmwf_w56_val_corrected.mean(dim=['realization', 'lead_time'])
ecmwf_w56_val_corrected_std = ecmwf_w56_val_corrected.std(dim=['realization', 'lead_time'])

In [ ]:
val_probabilistic_forecast_w56 = make_probabilistic(ecmwf_w56_val_corrected, obs_w56_train)

In [ ]:
val_probabilistic_forecast_w56.isel(category=2, forecast_dayofyear=40).plot()

In [ ]:
loc_w56 = ecmwf_w56_val_corrected_mean.compute().t2m
scale_w56 = ecmwf_w56_val_corrected_std.compute().t2m

In [ ]:
cdfs_w56 = xr.apply_ufunc(scipy.stats.norm.cdf, obs_w56_train_thresholds.t2m, dask='allowed', kwargs={'loc': loc, 'scale': scale})

In [ ]:
below_w56 = cdfs_w56.isel(quantile=0).drop_vars('quantile')
normal_w56 = (cdfs_w56.isel(quantile=1) - cdfs_w56.isel(quantile=0))
above_w56 = xr.ones_like(normal) - cdfs_w56.isel(quantile=1).drop_vars('quantile')

In [ ]:
val_probabilistic_obs_w56 = xr.concat([below_w56, normal_w56, above_w56], 'category').assign_coords(category=['below normal', 'near normal', 'above normal'])

In [ ]:
val_probabilistic_obs_w56.isel(category=2, forecast_dayofyear=0).plot()

In [ ]:
xs.rps(val_probabilistic_obs_w56, val_probabilistic_forecast, category_edges=None, input_distributions='p', dim=['latitude', 'longitude']).plot()

## Submission

### Example

In [ ]:
benchmark_path = sorted([f for f in pathlib.Path(BENCHNMARK).iterdir() if FIELD in f.stem])[-1]
benchmark_path

In [ ]:
benchmark_example = xr.open_mfdataset([benchmark_path], preprocess=fix_dataset_dims)
benchmark_example

### Ours

In [ ]:
val_probabilistic_obs_all = xr.concat([val_probabilistic_obs, val_probabilistic_obs_w56], "lead_time").to_dataset()

In [ ]:
val_probabilistic_obs_all = val_probabilistic_obs_all.assign_coords(lead_time=np.array([14, 28], dtype='timedelta64[D]'))

In [ ]:
val_probabilistic_obs_all.attrs = benchmark_example.attrs

In [ ]:
val_probabilistic_obs_all